<a href="https://colab.research.google.com/github/isacprogram/aprendizado-data/blob/main/Tratamento_de_dados_em_base_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.).

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
```
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'.
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [1]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape

# Não há duplicados

(27028, 69)


(27028, 69)

In [2]:
# 2) dados faltantes de todas as variáveis
miss = sinasc.isnull().sum().sum()
print(miss)

121594


In [4]:
# 3) selecionado as variáveis de meu interesse
df_i = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']].copy()
faltantes = df_i.isna().sum().sum()
print(f'dados faltantes: {faltantes}')
df_i


dados faltantes: 3616


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [5]:
# 4) verificando novamente os dados faltantes das variávei selecionadas
apgar_missing =  df_i['APGAR5'].isna().sum()
print(f' dados faltantes  em apgar 5 : {apgar_missing}')

 dados faltantes  em apgar 5 : 103


In [6]:
#4.1 apagando os dados faltantes
df_g = df_i.dropna(subset=['APGAR5'])

In [7]:
#4.2 verificando se foi apagado
print(df_g['APGAR5'].isna().sum())
df_g['APGAR5'].shape

0


(26925,)

In [8]:
# 5)  verificando os códigos
df_g[['ESTCIVMAE','CONSULTAS']]

,ESTCIVMAE,CONSULTAS
0,5.0,4
1,2.0,4
2,9.0,4
3,5.0,3
4,2.0,4
...,...,...
27023,2.0,4
27024,1.0,4
27025,2.0,4
27026,2.0,4


In [9]:
# 6) trocando os dados faltantes em quantidade de filhos por 0
mod1 = df_g['QTDFILVIVO'].copy()
mod = mod1.fillna(0)  # método que troca os dados faltantes por uma valor especificado no caso 0.
df_g['QTDFILVIVO'] = mod

<ipython-input-9-e282d62d4e8c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_g['QTDFILVIVO'] = mod


In [10]:
#6.1 verificando se a coluna foi modificada
print(df_g)
df_g['QTDFILVIVO'].isna().sum()

       LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO  \
0            1        19        5.0      8 a 11 anos         0.0   
1            1        29        2.0      8 a 11 anos         1.0   
2            1        37        9.0      8 a 11 anos         2.0   
3            1        30        5.0  12 anos ou mais         0.0   
4            1        30        2.0      8 a 11 anos         1.0   
...        ...       ...        ...              ...         ...   
27023        1        32        2.0  12 anos ou mais         1.0   
27024        1        19        1.0      8 a 11 anos         0.0   
27025        1        24        2.0      8 a 11 anos         0.0   
27026        1        21        2.0      8 a 11 anos         1.0   
27027        1        18        5.0      8 a 11 anos         1.0   

              GESTACAO GRAVIDEZ  CONSULTAS  APGAR5  
0      37 a 41 semanas    Única          4    10.0  
1      37 a 41 semanas    Única          4     9.0  
2      37 a 41 semanas  

0

In [11]:
#7 verificando dados faltantes em todas as variáveis
print(df_g['LOCNASC'].isna().sum())
print(df_g['IDADEMAE'].isna().sum())
print(df_g['ESTCIVMAE'].isna().sum())# contém dados faltantes
print(df_g['ESCMAE'].isna().sum())# contém dados faltantes
print(df_g['QTDFILVIVO'].isna().sum())
print(df_g['GESTACAO'].isna().sum())#contém dados faltantes
print(df_g['GRAVIDEZ'].isna().sum())#contém dados falntantes
print(df_g['CONSULTAS'].isna().sum())
print(df_g['APGAR5'].isna().sum())

0
0
315
310
0
1216
76
0
0


In [12]:
#7.1 indentificando as variáveis com dados faltantes para tratamento
print(df_g['ESTCIVMAE'].value_counts())# detalhe no dicionário de dados não contém o 5 como variável categórica!!!!
print(df_g['ESCMAE'].value_counts())
print(df_g['GESTACAO'].value_counts())
print(df_g['GRAVIDEZ'].value_counts())

2.0    9906
1.0    8907
5.0    7145
4.0     451
9.0     144
3.0      57
Name: ESTCIVMAE, dtype: int64
8 a 11 anos        15581
12 anos ou mais     5757
4 a 7 anos          4727
1 a 3 anos           510
Nenhuma               40
Name: ESCMAE, dtype: int64
37 a 41 semanas        22523
32 a 36 semanas         2234
42 semanas e mais        698
28 a 31 semanas          162
22 a 27 semanas           85
Menos de 22 semanas        7
Name: GESTACAO, dtype: int64
Única              26356
Dupla                488
Tríplice e mais        5
Name: GRAVIDEZ, dtype: int64


In [13]:
# pela proporção de quantidade de dados faltantes ser menos de 1 %.
# todos foram inseridos como 9  quer dizer ignorados.
proporcao_dados_faltantes = (df_g.isna().sum().sum()) / len(df_g)
print(round(proporcao_dados_faltantes,3))

df_g.loc[df_g['ESTCIVMAE'].isna(),'ESTCIVMAE']= df_g['ESTCIVMAE'].fillna(9.0)
print(df_g['ESTCIVMAE'].isna().sum())

df_g.loc[df_g['ESCMAE'].isna(), 'ESCMAE']= df_g['ESCMAE'].fillna(9.0)
print(df_g['ESCMAE'].isna().sum())

df_g.loc[df_g['GESTACAO'].isna(),'GESTACAO']= df_g['GESTACAO'].fillna(9.0)
print(df_g['GESTACAO'].isna().sum())

df_g.loc[df_g['GRAVIDEZ'].isna(),'GRAVIDEZ']= df_g['GRAVIDEZ'].fillna(9.0)
print(df_g['GRAVIDEZ'].isna().sum())



0.071
0
0
0
0


In [14]:
# 8) criando uma nova coluna para classificar o grau de asfixia sofrido pelo RN.
df_g.loc[(df_g['APGAR5'] >= 8.0) & (df_g['APGAR5'] <= 10.0), 'GRAU_DE_HIPOXIA'] = 'NORMAL'
df_g.loc[(df_g['APGAR5'] >= 6.0) & (df_g['APGAR5'] <= 7.0), 'GRAU_DE_HIPOXIA'] = 'ASFIXIA_LEVE'
df_g.loc[(df_g['APGAR5'] >= 4.0) & (df_g['APGAR5'] <= 5.0), 'GRAU_DE_HIPOXIA'] = 'ASFIXIA_MODERADA'
df_g.loc[(df_g['APGAR5'] >= 0.0) & (df_g['APGAR5'] <= 3.0), 'GRAU_DE_HIPOXIA'] = 'ASFIXIA_SEVERA'

df_g



<ipython-input-14-8b7b67007250>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_g.loc[(df_g['APGAR5'] >= 8.0) & (df_g['APGAR5'] <= 10.0), 'GRAU_DE_HIPOXIA'] = 'NORMAL'


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,GRAU_DE_HIPOXIA
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,NORMAL
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,NORMAL
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,NORMAL
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,NORMAL
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,NORMAL
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,NORMAL
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,NORMAL
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,NORMAL
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,NORMAL


In [15]:
#8.1 verificando os valores

df_g['GRAU_DE_HIPOXIA'].value_counts()

NORMAL              26463
ASFIXIA_LEVE          320
ASFIXIA_SEVERA         74
ASFIXIA_MODERADA       68
Name: GRAU_DE_HIPOXIA, dtype: int64

In [16]:
# 9) renomeando as colunas do data frame para padrão snake_case

local_nas = 'LOCNASC'
idade_mae = 'IDADEMAE'
esta_civil = 'ESTCIVMAE'
escolaridade = 'ESCMAE'
qtd_filho = 'QTDFILVIVO'
gestaçao = 'GESTACAO'
gravidez = 'GRAVIDEZ'
consultas = 'CONSULTAS'
apgar5 = 'APGAR5'
hipoxia = 'GRAU_DE_HIPOXIA'
colunas_renomeadas = {
    local_nas : local_nas.lower(),
    idade_mae : idade_mae.lower(),
    esta_civil : esta_civil.lower(),
    escolaridade : escolaridade.lower(),
    qtd_filho : qtd_filho.lower(),
    gestaçao : gestaçao.lower(),
    gravidez : gravidez.lower(),
    consultas : consultas.lower(),
    apgar5 : apgar5.lower(),
    hipoxia : hipoxia.lower()
}
df_g.rename(columns=colunas_renomeadas, inplace=True)
df_g


<ipython-input-16-756e98738368>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_g.rename(columns=colunas_renomeadas, inplace=True)


,locnasc,idademae,estcivmae,escmae,qtdfilvivo,gestacao,gravidez,consultas,apgar5,grau_de_hipoxia
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,NORMAL
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,NORMAL
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,NORMAL
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,NORMAL
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,NORMAL
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,NORMAL
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,NORMAL
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,NORMAL
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,NORMAL
